In [1]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

import pickle
import pandas as pd
import time

from utils import greedy, simplex_projection, integer_initial_vector
from continuous import solve_continuous_quadratic

In [2]:
obj = 'crash'
setting = 'n800'

results = []
for alpha in [0.01, 0.1, 1]:
    params = np.load(f'data/{obj}-{setting}/o{alpha}/params.npz')
    n = params['n']
    R = params['R']

    for trial in range(10):
        yinit = integer_initial_vector(n, R)
        y = integer_initial_vector(n, R)
        pred = integer_initial_vector(n, R)

        for t in range(100):
            with open(f'data/{obj}-{setting}/o{alpha}/{trial*100+t}.pkl', 'rb') as f:
                G = pickle.load(f)

            print('start!')

            # Cold
            start = time.time()
            itr_c, _ = greedy(yinit, G)
            time_c = time.time() - start

            print(time_c, itr_c)

            # Learn
            start = time.time()
            itr_p, x = greedy(pred, G)
            time_p = time.time() - start

            print(time_p, itr_p)

            # online subgradient descent update
            y += .01 * (R/np.sqrt(n)) * np.sign(x - y)
            y = simplex_projection(y, R)
            pred = (pred*(t+1) + y)/(t+2)

            # Relax            
            # qsol, time_qc = solve_continuous_quadratic(G)
            # start = time.time()
            # itr_q, _ = greedy(qsol, G)
            # time_qg = time.time() - start
            # time_q = time_qc + time_qg

            results.append([alpha, beta, trial, t, itr_c, itr_p, itr_q, time_c, time_p, time_q, time_qc, time_qg])

            print(alpha, beta, trial, t, itr_c, itr_p, itr_q, time_c, time_p, time_q, time_qc, time_qg)

# create a DataFrame from the results list
df = pd.DataFrame(results, columns=['alpha', 'beta', 'trial', 't', 'itr_c', 'itr_p', 'itr_q', 'time_c', 'time_p', 'time_q', 'time_qc', 'time_qg'])
#save df as pickle in result folder
df.to_pickle(f'result/{obj}-{setting}.pkl')

start!
27.997467279434204 4881
28.067294120788574 4892


NameError: name 'beta' is not defined